#### Attendee registration

In [1]:
import json
from ipynb.fs.full.gpt import OpenAI
attendees = json.load(open('mock-attendees.json'))

/Users/javier/.local/share/virtualenvs/ice-breaker-kYPj9F3u/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
openai = OpenAI(relevant_memory=[], full_message_history=[])

def generate_category_name(text):
    (_,
     _,
     _,
     current_context) = openai.generate_chat_context("""
        You are a help AI assistant for a conference. You are given a list of attendees
        and their interests. You need to create a category for each attendee based on
        their interests.
    """)
    current_context.append(openai.create_chat_message(role='user', content=text))
    return openai.create_chat_completion(current_context, 'gpt-4')
    

def clustering_by_interests(attendees):
    categories = []
    for attendee in attendees:
        interests = attendee['interests']
        category_name_prompt = f"""
            Create a single category name for a person with the following interests:
                {", ".join(interests)}
            If the category fits one of the categories below, use that instead:
                {", ".join(categories)}
            Only reply with the name of category. nothing else.
        """
        category_name = generate_category_name(category_name_prompt)
        categories.append(category_name)
        attendee['category'] = category_name
    return attendees
    

In [5]:
attendee_categories =  clustering_by_interests(attendees)
json.dump(attendee_categories, open('attendee-categories.json', 'w'), indent=2)

In [9]:
import pandas as pd

df = pd.DataFrame(attendee_categories)
by_category = df.sort_values(by='category')
category = ""
for _,row in by_category.iterrows():
    if category != row['category']:
        category = row['category']
        print("---")
    print(row['full_name'], "-", row['category'])


---
Avery Anderson - Data Science and AI
Chloe Adams - Data Science and AI
Lily Clark - Data Science and AI
Amelia Taylor - Data Science and AI
---
Leah Roberts - Emerging Technologies and AI
---
Zoe Thomas - Software and IoT Development
Scarlett White - Software and IoT Development
Emily Hall - Software and IoT Development
Natalie Jackson - Software and IoT Development
Ella Lewis - Software and IoT Development
Mia Wilson - Software and IoT Development
Isabella Davis - Software and IoT Development
Ava Williams - Software and IoT Development
Olivia Martin - Software and IoT Development
Harper Harris - Software and IoT Development
---
Charlotte Thompson - Technology and Security
Sophie Walker - Technology and Security
Grace Turner - Technology and Security
Sophia Brown - Technology and Security
Emma Johnson - Technology and Security
John Doe - Technology and Security
